In [2]:
import scanpy as sc
import squidpy as sq

import numpy as np
import pandas as pd

In [2]:
adata4 = sc.read_h5ad('datasets/merfish/H20.30.001.STG.4000.expand.rep1.h5ad')
adata4.obs['coord'] = np.sqrt((adata4.obsm['spatial'][:,0]+3500) ** 2 + (adata4.obsm['spatial'][:,1]-2000) ** 2)
adata4.obs['niche'] = 'N6'
adata4.obs['niche'][adata4.obs['coord']> 3500] = 'N5'
adata4.obs['niche'][adata4.obs['coord']> 4700] = 'N4'
adata4.obs['niche'][adata4.obs['coord']> 5600] = 'N3'
adata4.obs['niche'][adata4.obs['coord']> 7000] = 'N2'
adata4.obs['niche'][adata4.obs['coord']> 7900] = 'N1'

adata5 = sc.read_h5ad('datasets/merfish/H20.30.001.STG.4000.expand.rep2.h5ad')
adata5.obs['coord'] = -((adata5.obsm['spatial'][:,0]+3500) + 0.4 * (adata5.obsm['spatial'][:,1]-2000)) +5000
adata5.obs['coord2'] = -((adata5.obsm['spatial'][:,0]+3500) + 0.3 * (adata5.obsm['spatial'][:,1]-2000)) +5000
adata5.obs['niche'] = 'N6'
adata5.obs['niche'][adata5.obs['coord']> 1800] = 'N5'
adata5.obs['niche'][adata5.obs['coord2']> 3000] = 'N4'
adata5.obs['niche'][adata5.obs['coord2']> 3400] = 'N3'
adata5.obs['niche'][adata5.obs['coord']> 5200] = 'N2'
adata5.obs['niche'][adata5.obs['coord']> 5600] = 'N1'

adata6 = sc.read_h5ad('datasets/merfish/H20.30.001.STG.4000.expand.rep3.h5ad')
adata6.obs['coord'] = adata6.obsm['spatial'][:,1]
adata6.obs['niche'] = 'N6'
adata6.obs['niche'][adata6.obs['coord']> 2500] = 'N5'
adata6.obs['niche'][adata6.obs['coord']> 4000] = 'N4'
adata6.obs['niche'][adata6.obs['coord']> 4400] = 'N3'

adata4.obs['batch'] = 'H20.30.001.STG.4000.rep1'
adata5.obs['batch'] = 'H20.30.001.STG.4000.rep2'
adata6.obs['batch'] = 'H20.30.001.STG.4000.rep3'

adata_stg = sc.concat([adata4,adata5,adata6])
adata_stg.obs_names_make_unique()
adata_stg

/tmp/ipykernel_646484/481034249.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata4.obs['niche'][adata4.obs['coord']> 3500] = 'N5'
/tmp/ipykernel_646484/481034249.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata4.obs['niche'][adata4.obs['coord']> 4700] = 'N4'
/tmp/ipykernel_646484/481034249.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata4.obs['niche'][adata4.obs['coord']> 5600] = 'N3'
/tmp/ipykernel_646484/481

AnnData object with n_obs × n_vars = 14924 × 4000
    obs: 'name', 'global.x', 'global.y', 'adjusted.x', 'adjusted.y', 'fov.x', 'fov.y', 'cluster_L1', 'cluster_L2', 'cluster_L3', 'coord', 'niche', 'batch'
    obsm: 'spatial'

In [3]:
# graphonly models

from simvi.model import SimVIGraphOnly

Global seed set to 0


In [4]:
SimVIGraphOnly.setup_anndata(adata_stg,batch_key='batch')
edge_index = SimVIGraphOnly.extract_edge_index(adata_stg,n_neighbors=10,batch_key='batch')

## likelihood

from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i)
    model = SimVIGraphOnly(adata_stg,kl_weight=1,kl_gatweight=1,lam_mi=500,permutation_rate=0,n_spatial=20)
    train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=500,use_gpu=True)
    adata_stg.obsm['simvigo_s_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
Global seed set to 0
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:45<00:00,  2.20it/s, train_loss=741, val_loss=783.71234]
Global seed set to 1
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:47<00:00,  2.13it/s, train_loss=744, val_loss=786.52734]
Global seed set to 2
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:45<00:00,  2.19it/s, train_loss=745, val_loss=784.38837]
Global seed set to 3
Epoch 100/

In [5]:
adata_stg.obs['batch_ct'] = adata_stg.obs['batch'].astype(str) + adata_stg.obs['cluster_L1'].astype(str)

SimVIGraphOnly.setup_anndata(adata_stg,batch_key='batch_ct')
edge_index = SimVIGraphOnly.extract_edge_index(adata_stg,n_neighbors=10,batch_key='batch')

from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i)
    model = SimVIGraphOnly(adata_stg,kl_weight=1,kl_gatweight=1,lam_mi=500,permutation_rate=0,n_spatial=20)
    train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=500,use_gpu=True)
    adata_stg.obsm['simvigoct_s_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)

Global seed set to 0
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:45<00:00,  2.20it/s, train_loss=726, val_loss=749.3489]
Global seed set to 1
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:47<00:00,  2.10it/s, train_loss=727, val_loss=748.3678]
Global seed set to 2
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:46<00:00,  2.13it/s, train_loss=728, val_loss=749.0781]
Global seed set to 3
Epoch 100/100: 100%|███████████████████████████████████████████████████████████████████████████████████

In [6]:
adata_stg.obs['batch_ct2'] = adata_stg.obs['batch'].astype(str) + adata_stg.obs['cluster_L2'].astype(str)

SimVIGraphOnly.setup_anndata(adata_stg,batch_key='batch_ct2')
edge_index = SimVIGraphOnly.extract_edge_index(adata_stg,n_neighbors=10,batch_key='batch')

from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i)
    model = SimVIGraphOnly(adata_stg,kl_weight=1,kl_gatweight=1,lam_mi=500,permutation_rate=0,n_spatial=20)
    train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=500,use_gpu=True)
    adata_stg.obsm['simvigoct2_s_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)

/usr/local/lib/python3.8/dist-packages/scvi/data/_utils.py:156: UserWarning: Category 21 in adata.obs['_scvi_batch'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
Global seed set to 0
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:46<00:00,  2.15it/s, train_loss=720, val_loss=744.14453]
Global seed set to 1
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:47<00:00,  2.13it/s, train_loss=720, val_loss=743.9392]
Global seed set to 2
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100

In [7]:
import scvi
scvi.model.SCVI.setup_anndata(adata_stg,batch_key='batch')
for i in range(10):
    seed_everything(i)
    vae = scvi.model.SCVI(adata_stg, n_layers=2, n_latent=20)
    vae.train(max_epochs=100,use_gpu=True,batch_size=500)
    adata_stg.obsm['scvi_'+str(i)] = vae.get_latent_representation(adata_stg)

Global seed set to 0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:40<00:00,  2.49it/s, loss=713, v_num=1]

Global seed set to 1


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:40<00:00,  2.49it/s, loss=715, v_num=1]

...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:39<00:00,  2.51it/s, loss=720, v_num=1]

Global seed set to 9


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:39<00:00,  2.51it/s, loss=718, v_num=1]


In [5]:
from simvi.model import SimVI
SimVI.setup_anndata(adata_stg,batch_key='batch')
edge_index = SimVI.extract_edge_index(adata_stg,n_neighbors=10,batch_key='batch')
from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i)
    model = SimVI(adata_stg,kl_weight=1,kl_gatweight=0.01,lam_mi=1000,permutation_rate=0.5,n_spatial=20,n_intrinsic=20)
    train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=500,use_gpu=True,mae_epochs=25)
    adata_stg.obsm['simvikl2_z'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
    adata_stg.obsm['simvikl2_lz'+str(i)] = model.get_latent_representation(edge_index,representation_kind='output',give_mean=True)
    adata_stg.obsm['simvikl2_s'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)

Global seed set to 0
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:28<00:00,  4.48s/it, train_loss=727, val_loss=746.05255]
Global seed set to 1
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:28<00:00,  4.49s/it, train_loss=728, val_loss=745.02673]
Global seed set to 2
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:56<00:00,  4.16s/it, train_loss=728, val_loss=745.9001]
Global seed set to 3
Epoch 100/100: 100%|███████████████████████████████████████████████████████████████████████████████████

In [19]:
adata_stg.write('stg_baselines.h5ad')